In [1]:
#function to test models 
import nest_asyncio
import warnings
from mb_utils.src.logging import logger

nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [2]:
from mb_pytorch.utils.version import version
version

'1.0.55'

In [65]:
from mb_pytorch.dataloader.loader_old import DataLoader

k = DataLoader('../scripts/detection/object_detection.yaml',logger=logger)
k_data = k.data_dict['data']
model_data_dict = k.data_dict['model']

Mon 22:42:03,231 INF Read data from yaml file: [{'data': {'from_datasets': 'CelebA', 'datasets_params_train': {'root': '/home/malav/D
Mon 22:42:03,233 INF Data folder already exists. Using existing data folder :  /home/malav/Desktop/mb_packages/mb_pytorch/data/      


In [66]:
from mb_pytorch.training.train_params import train_helper
model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [67]:
from torch.utils.tensorboard import SummaryWriter
import os

path_logs = os.path.join(k_data['datasets_params_train']['root'], 'logs')
writer = SummaryWriter(log_dir=path_logs)


In [68]:
train_loader,val_loader,train_dataset,test_dataset = k.data_load()


Mon 22:42:04,025 INF Data file: CelebA loading from torchvision.datasets.                                                            


Files already downloaded and verified
Files already downloaded and verified


In [69]:
train_dataset.__getitem__(4)

(tensor([[[0.5961, 0.5882, 0.5882,  ..., 0.5569, 0.5569, 0.5569],
          [0.5961, 0.5882, 0.5922,  ..., 0.5569, 0.5569, 0.5569],
          [0.5961, 0.5922, 0.5922,  ..., 0.5529, 0.5529, 0.5529],
          ...,
          [0.5725, 0.5725, 0.5725,  ..., 0.5686, 0.5529, 0.5529],
          [0.5686, 0.5725, 0.5725,  ..., 0.5686, 0.5725, 0.5725],
          [0.5686, 0.5725, 0.5725,  ..., 0.5686, 0.5804, 0.5804]],
 
         [[0.7725, 0.7725, 0.7765,  ..., 0.7922, 0.7922, 0.7922],
          [0.7725, 0.7725, 0.7804,  ..., 0.7922, 0.7922, 0.7922],
          [0.7725, 0.7765, 0.7804,  ..., 0.7961, 0.7961, 0.7961],
          ...,
          [0.7843, 0.7843, 0.7843,  ..., 0.7804, 0.7529, 0.7529],
          [0.7804, 0.7843, 0.7843,  ..., 0.7804, 0.7725, 0.7725],
          [0.7804, 0.7843, 0.7843,  ..., 0.7804, 0.7804, 0.7804]],
 
         [[0.7961, 0.7961, 0.7922,  ..., 0.7843, 0.7843, 0.7843],
          [0.7961, 0.7961, 0.7961,  ..., 0.7843, 0.7843, 0.7843],
          [0.7961, 0.8000, 0.7961,  ...,

In [70]:
for batch_idx, (images, targets) in enumerate(train_loader):
    print("image is : {}".format(images.shape))
    images_l = list(image.to('cpu') for image in images) 
    print("target is : {}".format(targets))
    d = {}
    d['boxes'] = targets[1][:]
    d['labels'] = targets[0][:]
    targets = [{k: v.to('cpu') for k, v in t.items()} for t in [d]]
    #print("image_new is : {}".format(images_l))
    print("target_new is : {}".format(targets))
    break

image is : torch.Size([4, 3, 218, 178])
target is : [tensor([2880, 2937, 8692, 5805]), tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]])]
target_new is : [{'boxes': tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]]), 'labels': tensor([2880, 2937, 8692, 5805])}]


In [71]:
targets[0]['labels']

tensor([2880, 2937, 8692, 5805])

In [72]:
from mb_pytorch.models.modelloader import ModelLoader
model_data_load = ModelLoader(k.data_dict['model'])
model =  model_data_load.get_model()

Mon 22:42:21,478 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   


In [73]:
targets[0]

{'boxes': tensor([[ 95,  71, 226, 313],
         [ 72,  94, 221, 306],
         [216,  59,  91, 126],
         [622, 257, 564, 781]]),
 'labels': tensor([2880, 2937, 8692, 5805])}

In [74]:
targets[0]

{'boxes': tensor([[ 95,  71, 226, 313],
         [ 72,  94, 221, 306],
         [216,  59,  91, 126],
         [622, 257, 564, 781]]),
 'labels': tensor([2880, 2937, 8692, 5805])}

In [75]:
for target in targets:
    print(target['boxes'])
    break

tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]])


In [76]:
model(images_l,targets)

IndexError: list index out of range

In [ ]:
images.shape

torch.Size([4, 3, 218, 178])

In [ ]:
#tuple(targets)
lk = [{k: v for k, v in t.items()} for t in tuple(targets)]
lk

[{'boxes': tensor([[ 95,  71, 226, 313],
          [ 72,  94, 221, 306],
          [216,  59,  91, 126],
          [622, 257, 564, 781]]),
  'labels': tensor([2880, 2937, 8692, 5805])}]

In [ ]:
target[i] if target[0] is not None else None

KeyError: 0

In [ ]:
len(images_l)

4

In [ ]:
boxes_list = [target['boxes'] for target in targets]
boxes_list

[tensor([[ 95,  71, 226, 313],
         [ 72,  94, 221, 306],
         [216,  59,  91, 126],
         [622, 257, 564, 781]])]

In [ ]:
boxes_label = [target['labels'] for target in targets]
boxes_label

[tensor([2880, 2937, 8692, 5805])]

In [ ]:
boxes_list[0][0]

tensor([ 95,  71, 226, 313])

In [ ]:
boxes_label[0][0]

tensor(2880)

In [ ]:
for i in range(len(boxes_list[0][0].tolist())):
    print(boxes_list[0][1].tolist()[i])


72
94
221
306


In [ ]:
boxes_list[0][1].tolist()

[72, 94, 221, 306]

In [ ]:
import torch
lk_new = []
for i in range(len(images_l)):
    temp_dict = {}
    # temp_l = []
    # temp_l_final = []
    # for i in range(len(boxes_list[0][0])):
    #     temp_l.append(int(boxes_list[0][i].tolist()[i]))
    #     temp_l_final.append(temp_l)
    temp_dict['boxes'] = torch.tensor([boxes_list[0][i].tolist()])
    temp_dict['labels'] = torch.tensor([boxes_label[0][i]])
    lk_new.append(temp_dict)
lk_new

[{'boxes': tensor([[ 95,  71, 226, 313]]), 'labels': tensor([2880])},
 {'boxes': tensor([[ 72,  94, 221, 306]]), 'labels': tensor([2937])},
 {'boxes': tensor([[216,  59,  91, 126]]), 'labels': tensor([8692])},
 {'boxes': tensor([[622, 257, 564, 781]]), 'labels': tensor([5805])}]

In [64]:
model(images_l,lk_new)

AssertionError: All bounding boxes should have positive height and width. Found invalid box [970.7864990234375, 264.9587097167969, 408.9887390136719, 565.8440551757812] for target at index 2.

In [19]:
from mb_pytorch.detection.training import detection_train_loop


In [20]:
detection_train_loop(k,model_data_dict,writer=writer,logger=logger)

Mon 21:26:41,084 INF Training loop Starting                                                                                          
Mon 21:26:41,499 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   
Mon 21:26:41,501 INF Model Loaded                                                                                                    
Mon 21:26:41,502 INF Data file: CelebA loading from torchvision.datasets.                                                            


Files already downloaded and verified
Files already downloaded and verified


Mon 21:26:58,013 INF Optimizer and Scheduler Loaded                                                                                  
Mon 21:26:58,014 INF Loss: <function cross_entropy at 0x7ce3255bcfe0>                                                                
Mon 21:26:58,016 INF Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: [0.9, 0.999]
    capturable: False
    differe
Mon 21:26:58,018 INF Scheduler: <torch.optim.lr_scheduler.StepLR object at 0x7ce311c92610>                                           
  0%|          | 0/10 [00:00<?, ?it/s]


IndexError: list index out of range